## 视频处理

In [1]:
import cv2
import numpy as np
from  matplotlib import pyplot as plt
%matplotlib inline


def cvshow(img):
    cv2.imshow('img', img)
    cv2.waitKey(10)
    cv2.destroyAllWindows()

def show(img):
    if img.ndim == 2:
        plt.imshow(img,cmap="gray")
    elif img.ndim == 3:
        plt.imshow(img)
    plt.show()
    
cap = cv2.VideoCapture('test.avi') #0或者其他数字代表摄像头ID
#------------视频差分运动检测
ret, frame0 = cap.read()
gray0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
while True:
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        dif = cv2.absdiff(gray,gray0)
        gray0 = gray
        cv2.imshow('img', dif)
        if cv2.waitKey(50) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    else:
        cv2.destroyAllWindows()
        break

### 统计平均背景建模

In [12]:
import pdb,cv2
import numpy as np
#-----------背景建模和差分
cap = cv2.VideoCapture('test.avi') #0或者其他数字代表摄像头ID
ret, frame0 = cap.read()
gray0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
hist_frames = []
hist_frames.append(gray0)
while True:
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist_frames.append(gray)
        if len(hist_frames)==100:
#             pdb.set_trace()
            np_frames = np.array(hist_frames)
            np_frames.shape
            bg = np.mean(np_frames, axis = 0)
            dif = cv2.absdiff(gray,bg.astype(np.uint8))
#             cv2.imshow('img', dif)
            _,thresh_frame = cv2.threshold(dif,80, 255, cv2.THRESH_BINARY)
#             cv2.imshow('img', thresh_frame)
            contours, hierarchy = cv2.findContours(thresh_frame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) 
            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)
                if w>50 and h>50:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)                
            cv2.imshow('img', frame)
            if cv2.waitKey(30) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
            hist_frames.pop(0)
    else:
        cv2.destroyAllWindows()
        break

### 混合高斯背景建模

In [11]:
import pdb,cv2
import numpy as np
#-----------背景建模和差分
cap = cv2.VideoCapture('test.avi') #0或者其他数字代表摄像头ID
ret, frame0 = cap.read()
gray0 = cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
backSub = cv2.createBackgroundSubtractorMOG2()
while True:
    ret, frame = cap.read()
    if ret:
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fgMask = backSub.apply(frame)
#         _,thresh_frame = cv2.threshold(dif,128, 255, cv2.THRESH_BINARY)
#         cv2.imshow('img', fgMask)
        contours, hierarchy = cv2.findContours(fgMask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) 
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            if w>50 and h>50:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)                
        cv2.imshow('img', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    else:
        cv2.destroyAllWindows()
        break

### 目标跟踪